In [7]:
import requests
import json
from selenium import webdriver
from time import sleep
from tqdm import tqdm

In [2]:
# setup api secrets
with open('../secrets/tiktok.json') as f:
    secrets = json.load(f)
    
client_key = secrets['client_key']
client_secret = secrets['client_secret']

# get access token
url = "https://open.tiktokapis.com/v2/oauth/token/"
headers = {
    "Content-Type": "application/x-www-form-urlencoded",
    "Cache-Control": "no-cache"
}
data = {
    "client_key": client_key,
    "client_secret": client_secret,
    "grant_type": "client_credentials"
}
response = requests.post(url, headers=headers, data=data)
response_json = response.json()
access_token = response_json['access_token']

In [3]:
def scrape_page(start_date, end_date, hashtag=None, max_count=100, search_id=None, cursor=None, 
                create_date=None, username=None, region_code=None, video_id=None, keyword=None, 
                music_id=None, effect_id=None, video_length=None):
    fields = "id,username,create_time,video_description,hashtag_names,view_count,is_stem_verified,voice_to_text"
    url = f"https://open.tiktokapis.com/v2/research/video/query/?fields={fields}"
    headers = {
        "authorization": f"bearer {access_token}"
    }
    query_conditions = []

    if keyword:
        query_conditions.append({ "operation": "EQ", "field_name": "keyword", "field_values": [keyword] })
    if hashtag:
        query_conditions.append({ "operation": "EQ", "field_name": "hashtag_name", "field_values": [hashtag] })
    if create_date:
        query_conditions.append({ "operation": "EQ", "field_name": "create_date", "field_values": [create_date] })
    if username:
        query_conditions.append({ "operation": "EQ", "field_name": "username", "field_values": [username] })
    if region_code:
        query_conditions.append({ "operation": "EQ", "field_name": "region_code", "field_values": [region_code] })
    if video_id:
        query_conditions.append({ "operation": "EQ", "field_name": "video_id", "field_values": [video_id] })
    if music_id:
        query_conditions.append({ "operation": "EQ", "field_name": "music_id", "field_values": [music_id] })
    if effect_id:
        query_conditions.append({ "operation": "EQ", "field_name": "effect_id", "field_values": [effect_id] })
    if video_length:
        query_conditions.append({ "operation": "EQ", "field_name": "video_length", "field_values": [video_length] })

    data = { 
        "query": {
            "and": query_conditions
        }, 
        "start_date": start_date,
        "end_date": end_date,
        "max_count": max_count
    }

    if search_id:
        data['search_id'] = search_id
    if cursor:
        data['cursor'] = cursor

    response = requests.post(url, headers=headers, json=data)
    
    if response.status_code != 200:
        raise Exception(f"Error: {response.status_code} - {response.text}")

    return response.json()

In [ ]:
start_date = '20240501'
end_date = '20240531'

scrape_kwargs = {
    'start_date': start_date,
    'end_date': end_date,
    'hashtag': 'booktok',
    'keyword': 'stitch with'
}
response = scrape_page(**scrape_kwargs)
videos = response['data']['videos']

while response['data']['has_more']:
    sleep(10)

    search_id = response['data']['search_id']
    cursor = response['data']['cursor']
    response = scrape_page(search_id=search_id, cursor=cursor, **scrape_kwargs)
    videos += response['data']['videos']

In [5]:
# with open('booktok.json', 'r') as f:
#     videos = json.load(f)

In [17]:
driver = webdriver.Firefox()

stitchers = []
stitchees = []
with open('booktok_edgelist.txt', 'w') as f:
    for idx in tqdm(range(len(videos))):
        video = videos[idx]
        url = f'https://www.tiktok.com/@{video["username"]}/video/{video["id"]}'

        driver.get(url)
        sleep(2)

        xpath = '/html/body/div[1]/div[2]/div[2]/div/div[2]/div[1]/div[1]/div[2]/div[2]/div[1]/div/h1/a[2]'
        links = driver.find_elements('xpath', xpath)
        hrefs = [link.get_attribute('href') for link in links]

        if len(hrefs) == 0:
            continue

        if len(hrefs) > 1:
            print(f'Multiple stitches found for {url}')
            print(hrefs)

        stitchers.append(url)
        stitchees.append(hrefs)
        f.write(f'{url}\t{hrefs[0]}\n')
        print(idx, f'{url}\t{hrefs[0]}\n')
        sleep(2)

driver.close()


0 https://www.tiktok.com/@alexandrealechelle/video/7375309720908827946	https://www.tiktok.com/@a.aliiiyha/video/7370388924633599274



1 https://www.tiktok.com/@breezingthroughbooks/video/7375302277772381483	https://www.tiktok.com/@kaceybarnes/video/7374563222675819819



2 https://www.tiktok.com/@leahs.library8/video/7375300736453758241	https://www.tiktok.com/@thatguyjae/video/7367479745308527915



3 https://www.tiktok.com/@bookbroke/video/7375298449333390638	https://www.tiktok.com/@honestteachervibes/video/7374942271126506795



4 https://www.tiktok.com/@jcarrwrites/video/7375298215614205226	https://www.tiktok.com/@authormkmorris/video/7374775496187317550



...

204 https://www.tiktok.com/@marissam777/video/7374629369802657067	https://www.tiktok.com/@ajintheafternoon/video/7368546778699812139



205 https://www.tiktok.com/@iwritesmutnotrealities/video/7374628173562006826	https://www.tiktok.com/@andr3wsky/video/7374468480009030954



206 https://www.tiktok.com/@sreyavt/video/7374626197105429776	https://www.tiktok.com/@sreyavt/video/7374294236901969169



  7%|▋         | 207/2986 [20:23<4:33:50,  5.91s/it]


KeyboardInterrupt: 